In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

from MiDaS.depth_loss import compute_depth_loss
from yolo_bbox_decoder.yolo_loss import compute_loss

# garbage collection
import gc

mixed_precision = True
try:  # Mixed precision training https://github.com/NVIDIA/apex
    from apex import amp
except:
    # print('Apex recommended for mixed precision and faster training: https://github.com/NVIDIA/apex')
    mixed_precision = False  # not installed

from main_model import OpNet

In [10]:
from dataset import TSAIDataset, get_device

train_dataset = TSAIDataset("D:\\data\\customdata\\train.txt",(416, 416),
                                                      is_training=True,data_dir_path = "D:\\data\\customdata\\images\\")

test_dataset = TSAIDataset("D:\\data\\customdata\\test.txt",(416, 416),
                                                      is_training=False,data_dir_path = "D:\\data\\customdata\\images\\")

In [11]:
next(iter(train_dataset))

{'image': tensor([[[0.67843, 0.68235, 0.68235,  ..., 0.75686, 0.74510, 0.74118],
          [0.67843, 0.68235, 0.68627,  ..., 0.75686, 0.74902, 0.74902],
          [0.68235, 0.68627, 0.68627,  ..., 0.76078, 0.76078, 0.76078],
          ...,
          [0.19608, 0.19608, 0.19608,  ..., 0.19608, 0.19608, 0.19608],
          [0.19608, 0.19608, 0.19608,  ..., 0.19608, 0.19608, 0.19608],
          [0.19608, 0.19608, 0.19608,  ..., 0.19608, 0.19608, 0.19608]],
 
         [[0.63922, 0.64314, 0.64314,  ..., 0.74902, 0.73725, 0.73333],
          [0.63922, 0.64314, 0.64706,  ..., 0.74902, 0.74118, 0.74118],
          [0.64314, 0.64706, 0.64706,  ..., 0.75294, 0.75294, 0.75294],
          ...,
          [0.20000, 0.20000, 0.20000,  ..., 0.20000, 0.20000, 0.20000],
          [0.20000, 0.20000, 0.20000,  ..., 0.20000, 0.20000, 0.20000],
          [0.20000, 0.20000, 0.20000,  ..., 0.20000, 0.20000, 0.20000]],
 
         [[0.63529, 0.63922, 0.63922,  ..., 0.75294, 0.74118, 0.73725],
          [0.63529,

In [12]:
train_dataloader = torch.utils.data.DataLoader(train_dataset,
                                          batch_size=32,
                                          shuffle=True, num_workers=4, pin_memory=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset,
                                          batch_size=32,
                                          shuffle=True, num_workers=4, pin_memory=True)

In [13]:
# for sample in test_dataloader:
    
#     print(sample['image'].shape)
#     break

torch.Size([32, 3, 416, 416])


### ARGS

In [6]:
import sys

yolo_args = {
   'batch_size' : 1,
    'cfg' : './yolo_bbox_decoder/cfg/yolov3-custom.cfg',
    'epochs' : 10,
    'weights' : './yolov3-spp-ultralytics.pt',
    'data' : 'data/customdata/custom.data' ,
    'img_size' : [512,512,512] #[640,640,640]
}


midas_args = {
    'input' :'./data/customdata/images',
    'weights' : 'midas/model-f6b98070.pt'
}


loss_weightage = {
    
    'yolo' : 1,
    'midas' : 1
}

from options import *
import sys
sys.argv = ['']
yolo_args = yolo_parse_args()
yolo_args.batch_size = 1
yolo_args.cfg = 'yolo_bbox_decoder/cfg/yolov3-custom.cfg'
yolo_args.epochs = 10
yolo_args.weights = './yolov3-spp-ultralytics.pt'
yolo_args.data = './data/customdata/custom.data'
yolo_args.img_size=[512,512,512]

In [7]:
opt = yolo_args
cfg = opt.cfg

###  Model

In [15]:
from main_model import OpNet

device = get_device(force_cpu=False)

model = OpNet(yolo_cfg = yolo_args.cfg, midas_cfg = midas_args).to(device)

use_pretrained True
path midas/model-f6b98070.pt


Using cache found in C:\Users\saina/.cache\torch\hub\facebookresearch_WSL-Images_master


Model Summary: 582 layers, 1.70705e+08 parameters, 1.70705e+08 gradients


## Training

In [16]:
%matplotlib inline
from train import train

train(model, train_dataloader, device, yolo_args, midas_args, loss_weightage, model_save_path='./')

ModuleNotFoundError: No module named 'model'